# Observing conditions check

> Check that the observing conditions meet requirements.

In [ ]:
# |default_exp diagnostics.obs_cond_check

In [ ]:
# |export


from qagmire.data import (
    get_lr_l1_single_files,
    read_primary_header,
)
from qagmire.quality_assurance import Diagnostics
from qagmire.utilities import parse_obstemp

To write checks of the data, we create a subclass of `Diagnostics` and implement the `tests` method.

In [ ]:
# |export


class ObsCondCheck(Diagnostics):
    """Observing conditions check.

    A reproduction of the weaveio [obs_cond_check](https://github.com/bamford/QAG/blob/master/diagnostics/obs_cond_checks.py).

    This tests for the following cases:

    * Is the sky brighter than the requirement?
    * Is the seeing worse than the requirement?

    and also some supplementary tests:

    * Are there the other than two runs with the same MJD?
    * Do runs with the same MJD have different sky brightness?
    * Do runs with the same MJD have different seeing?
    """

    def __init__(
        self,
        sky_tolerance: float = 0.0,  # the tolerance in the sky brightness in magnitudes
        seeing_tolerance: float = 0.0,  # the tolerance in the seeing in arcsec
        by_exposure=False,  # should the checks be performed per exposure, or per OB (the default)
    ):
        self.sky_tolerance = sky_tolerance
        self.seeing_tolerance = seeing_tolerance
        if by_exposure:
            self._get_and_check = self._get_and_check_by_exp
        else:
            self._get_and_check = self._get_and_check_by_ob
        super().__init__()

    @staticmethod
    def _restore_coords(coords, da):
        return [d.assign_coords(coords) for d in da]

    @classmethod
    def _get_and_check_by_exp(cls, col):
        coords = (
            col.swap_dims(filename="MJD")
            .coords.to_dataset()
            .reset_coords()
            .groupby("MJD")
            .first()
        )
        by_exp = col.groupby("MJD")
        count, first, last = cls._restore_coords(
            coords, (by_exp.count(), by_exp.first(), by_exp.last())
        )
        expected_runs = count == 2
        runs_match = first == last
        return first, expected_runs, runs_match

    @staticmethod
    def _get_and_check_by_ob(col):
        by_ob = col.groupby("OBID")
        count, first = (by_ob.count(), by_ob.first())
        expected_runs = count == 6
        runs_match = (first != col).any(axis=-1)
        return first, expected_runs, runs_match

    def tests(
        self,
        **kwargs,
    ):
        files = get_lr_l1_single_files(**kwargs)
        hdr = read_primary_header(files)

        obstemp, two_runs, obstemp_runs_match = self._get_and_check(hdr["OBSTEMP"])
        obs = parse_obstemp(obstemp)

        sky, _, sky_runs_match = self._get_and_check(hdr["SKYBRTEL"])
        sky_fail = sky < obs["sky_brightness"] - self.sky_tolerance
        seeing, _, seeing_runs_match = self._get_and_check(hdr["SEEINGB"])
        seeing_fail = seeing > obs["seeing"] + self.seeing_tolerance

        tests = [
            {
                "name": "sky_too_bright",
                "description": "Is the sky brighter than the requirement?",
                "test": ~sky_fail,
            },
            {
                "name": "seeing_too_poor",
                "description": "Is the seeing worse than the requirement?",
                "test": ~seeing_fail,
            },
            {
                "name": "wrong_run_count",
                "description": "Are there the other than six runs in each OB?",
                "test": ~two_runs,
            },
            {
                "name": "unmatched_runs_sky",
                "description": "Do runs in the same OB have different sky brightness?",
                "test": ~sky_runs_match,
            },
            {
                "name": "unmatched_runs_seeing",
                "description": "Do runs in the same OB have different seeing?",
                "test": ~seeing_runs_match,
            },
        ]
        return tests

In [ ]:
tests = ObsCondCheck()
tests.run(date="201*")

Reading files: 100%|██████████| 126/126 [00:01<00:00, 91.54it/s]
Creating Dataset... took 2.23 s. Size is 0.796 Mb
Tests took 0.00 s to perform.
sky_too_bright:
    Is the sky brighter than the requirement?
seeing_too_poor:
    Is the seeing worse than the requirement?
wrong_run_count:
    Are there the other than six runs in each OB?
unmatched_runs_sky:
    Do runs in the same OB have different sky brightness?
unmatched_runs_seeing:
    Do runs in the same OB have different seeing?


In [ ]:
tests.summary_per_test()

,total fails
OBID,
test,
seeing_too_poor,21
sky_too_bright,19
wrong_run_count,0
unmatched_runs_sky,0
unmatched_runs_seeing,0


In [ ]:
tests.summary()

failed                 total fails
test sky_too_bright seeing_too_poor            
OBID                                           
3133           True            True           2
3170           True            True           2
3175           True            True           2
3189           True            True           2
3191           True            True           2
3295           True            True           2
3346           True            True           2
3372           True            True           2
3380           True            True           2
3756           True            True           2
3653           True            True           2
3803           True            True           2
3802           True            True           2
3936           True            True           2
4321           True            True           2
3806           True            True           2
3900           True            True           2
4407           True            True           2
4464           True            True           2
3434          False            True           1
3217          False            True           1

In [ ]:
tests.full_summary()

failed                                                     \
test sky_too_bright seeing_too_poor wrong_run_count unmatched_runs_sky   
OBID                                                                     
3133           True            True           False              False   
3170           True            True           False              False   
3175           True            True           False              False   
3189           True            True           False              False   
3191           True            True           False              False   
3217          False            True           False              False   
3295           True            True           False              False   
3346           True            True           False              False   
3372           True            True           False              False   
3380           True            True           False              False   
3434          False            True           False              False   
3653           True            True           False              False   
3756           True            True           False              False   
3802           True            True           False              False   
3803           True            True           False              False   
3806           True            True           False              False   
3900           True            True           False              False   
3936           True            True           False              False   
4321           True            True           False              False   
4407           True            True           False              False   
4464           True            True           False              False   

                            
test unmatched_runs_seeing  
OBID                        
3133                 False  
3170                 False  
3175                 False  
3189                 False  
3191                 False  
3217                 False  
3295                 False  
3346                 False  
3372                 False  
3380                 False  
3434                 False  
3653                 False  
3756                 False  
3802                 False  
3803                 False  
3806                 False  
3900                 False  
3936                 False  
4321                 False  
4407                 False  
4464                 False

In [ ]:
# |hide
import nbdev

nbdev.nbdev_export()